<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/TOON_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install python-toon

In [1]:
!pip show toon

Name: toon
Version: 0.15.9
Summary: Tools for neuroscience experiments
Home-page: https://github.com/aforren1/toon
Author: Alexander Forrence
Author-email: alex.forrence@gmail.com
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, psutil
Required-by: 


In [5]:
!pip install google-genai

In [6]:
import os
# ❌ OLD: import google.generativeai as genai
# ✅ NEW: Import the core client from the top-level 'google' namespace
from google import genai
from google.genai import types # Recommended for structured data/tool use

from toon import encode, decode
from typing import Dict, Any, List
import json

# ... (AgentConfig and API Key setup remains the same) ...

client = None
if GOOGLE_API_KEY:
    try:
        # The 'Client' is now accessed directly under the alias 'genai'
        client = genai.Client(api_key=GOOGLE_API_KEY)
        print(f"Gemini client initialized for model: {AgentConfig.LLM_MODEL_NAME}")
    except Exception as e:
        # This block should now correctly initialize the client
        print(f"Error during client initialization: {e}")
        client = None
else:
    print("FATAL: Gemini API Key not found. Cannot run integration demo.")
    client = None

# ... (The rest of your code) ...

Gemini client initialized for model: gemini-1.5-flash


In [8]:
import os
# CORRECTED IMPORT: Use 'from google import genai' for the modern SDK
from google import genai
from google.genai import types

# Assuming 'python-toon' is installed: pip install python-toon
from toon import encode, decode
from typing import Dict, Any, List
import json

# --- 1. Google Gemini API Configuration (Model Name Updated) ---
class AgentConfig:
    # --- FIX: Updated the model name to the current stable alias ---
    LLM_MODEL_NAME: str = "gemini-2.5-flash"

GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    # Fallback to environment variable
    GOOGLE_API_KEY = os.getenv('GEMINI')

client = None
if GOOGLE_API_KEY:
    try:
        # CORRECTED CLIENT INITIALIZATION: The Client object is now directly under the 'genai' alias
        client = genai.Client(api_key=GOOGLE_API_KEY)
        print(f"Gemini client initialized for model: {AgentConfig.LLM_MODEL_NAME}")
    except Exception as e:
        print(f"Error during client initialization: {e}")
        client = None
else:
    print("FATAL: Gemini API Key not found. Cannot run integration demo.")
    client = None

# ----- 2. Define Sample Data -----
SALES_DATA: List[Dict[str, Any]] = [
    {"Region": "East", "Sales_Amt": 45000, "Qtr": 1, "Status": "Target Met"},
    {"Region": "West", "Sales_Amt": 62000, "Qtr": 1, "Status": "Target Exceeded"},
    {"Region": "East", "Sales_Amt": 38000, "Qtr": 2, "Status": "Under Target"},
    {"Region": "West", "Sales_Amt": 75000, "Qtr": 2, "Status": "Target Exceeded"},
]

# ----- 3. Core TOON Integration Function -----
def analyze_data_with_gemini(data: List[Dict[str, Any]], analysis_prompt: str):
    """
    Encodes the Python list of dicts into TOON format and sends it to Gemini
    for analysis.
    """
    if not client:
        print("Integration failed: Gemini client is not initialized.")
        return

    # A. Encode the structured data into the token-efficient TOON format
    data_to_encode = {"sales_records": data}
    toon_data: str = encode(data_to_encode)

    # B. Output efficiency metrics
    print("--------------------------------------------------")
    print(f"Original JSON data size: {len(json.dumps(data_to_encode))} characters")
    print(f"TOON Data (sent to Gemini): {len(toon_data)} characters")
    print(f"TOON String:\n{toon_data}")
    print("--------------------------------------------------")

    # C. Construct the full prompt
    full_prompt = f"""
    Analyze the following data, which is formatted using TOON (Token-Oriented Object Notation).

    TOON Data:
    {toon_data}

    Analysis Request:
    {analysis_prompt}
    """

    # D. Call the Gemini API
    print(f"Sending prompt to {AgentConfig.LLM_MODEL_NAME}...")
    try:
        response = client.models.generate_content(
            model=AgentConfig.LLM_MODEL_NAME,
            contents=[full_prompt]
        )

        # E. Display the result
        print("\n✅ Gemini Analysis Result:")
        print(response.text)

    except Exception as e:
        print(f"\n❌ An error occurred during the Gemini API call: {e}")

# ----- 4. Execution -----
if __name__ == "__main__":

    # Define the specific task for the LLM
    task = "Calculate the total sales amount for the 'West' region across both quarters, and provide a single-sentence summary of the overall status trend."

    # Run the integration
    analyze_data_with_gemini(
        data=SALES_DATA,
        analysis_prompt=task
    )

Google Generative AI configured successfully using Colab Secrets.
Gemini client initialized for model: gemini-2.5-flash
--------------------------------------------------
Original JSON data size: 327 characters
TOON Data (sent to Gemini): 163 characters
TOON String:
sales_records[4,]{Region,Sales_Amt,Qtr,Status}:
  East,45000,1,Target Met
  West,62000,1,Target Exceeded
  East,38000,2,Under Target
  West,75000,2,Target Exceeded
--------------------------------------------------
Sending prompt to gemini-2.5-flash...

✅ Gemini Analysis Result:
Total sales for the 'West' region across both quarters is $137,000 (62,000 + 75,000). The overall status trend shows a mixed performance, with the West region consistently exceeding targets, while the East region's performance declined from meeting to falling under target.
